In [ ]:
!pip install -U ensemble-boxes

In [ ]:
import pandas as pd
import numpy as np
from ensemble_boxes import *
from glob import glob
import copy
from tqdm import tqdm
import shutil

In [ ]:
height_dict = pd.read_csv('../input/vinbigdata-original-image-dataset/vinbigdata/test.csv').to_dict('records')
fnl_dict ={}
for ix,i in enumerate(height_dict):
    fnl_dict[i['image_id']] = [i['width'],i['height'],i['width'],i['height']]

In [ ]:
subs = [
#     pd.read_csv('../input/submission-vbg/ds_tst_F2_noTH_noClean.csv'),
#         pd.read_csv('../input/vbg-yolo-submission/Fold 1.csv'),
#         pd.read_csv('../input/detectron2-resnet50/results/20210110_train_all_500k_512/submission_det.csv'),
#         pd.read_csv('../input/submission-vbg/ds_tst_F1_noTH_noClean.csv'),
#         pd.read_csv('../input/vbg-yolo-submission/Fold 4.csv'),
#         pd.read_csv('../input/vbg-yolo-submission/submission (2).csv'),
#         pd.read_csv('../input/modified-effnet-classification/submission (6).csv'),
#         pd.read_csv('../input/modified-effnet-classification/submission_v25_PS_kaggle_nbv5.csv'),
#         pd.read_csv('../input/modified-effnet-classification/submission (13).csv'),
#         pd.read_csv('../input/modified-effnet-classification/submission (4).csv'),
#         pd.read_csv('../input/modified-effnet-classification/submission (8).csv'),
#         pd.read_csv('../input/modified-effnet-classification/submission (19).csv')
#         pd.read_csv('../input/cv-yolo/submission (31).csv')
    
    pd.read_csv('../input/yolo-vbd-lots-of-decimals/Fold_1.csv'),
    pd.read_csv('../input/yolo-vbd-lots-of-decimals/Fold_2.csv'),
    pd.read_csv('../input/yolo-vbd-lots-of-decimals/Fold_3.csv'),
    pd.read_csv('../input/yolo-vbd-lots-of-decimals/Fold_4.csv'),
    pd.read_csv('../input/yolo-vbd-lots-of-decimals/Fold_5.csv'),
    pd.read_csv('../input/vinbigdata-cxr-ad-yolov5-14-class-infer-184dd1/submission.csv')
       ]

pred_2cls = pd.read_csv('../input/vinbigdata-2class-prediction/2-cls test pred.csv')

In [ ]:
def submission_decoder(df:pd.DataFrame) -> np.ndarray:
    info = df.values
    df_lst = []
    for i in info:
        pre_lst = i[1].split(' ')
        for j in range(0,len(pre_lst),6):
            df_lst.append([i[0],int(pre_lst[j]),float(pre_lst[j+1]),int(pre_lst[j+2]),int(pre_lst[j+3]),\
                           int(pre_lst[j+4]),int(pre_lst[j+5]),fnl_dict.get(i[0])[0],fnl_dict.get(i[0])[1]])
    return pd.DataFrame(df_lst,columns = ['image_id','class_id','score','x_min','y_min','x_max','y_max','width','height'])

In [ ]:
subs = [submission_decoder(subs[i]) for i in range(len(subs))]

In [ ]:
boxes_dict = {}
scores_dict = {}
labels_dict = {}
whwh_dict = {}

for i in tqdm(subs[0].image_id.unique()):
    if not i in boxes_dict.keys():
        boxes_dict[i] = []
        scores_dict[i] = []
        labels_dict[i] = []
        whwh_dict[i] = []

    size_ratio = fnl_dict.get(i)
    whwh_dict[i].append(size_ratio) 
    tmp_df = [subs[x][subs[x]['image_id']==i] for x in range(len(subs))]
    
    for x in range(len(tmp_df)):
        boxes_dict[i].append(((tmp_df[x][['x_min','y_min','x_max','y_max']].values)/size_ratio).tolist())
        scores_dict[i].append(tmp_df[x]['score'].values.tolist())
        labels_dict[i].append(tmp_df[x]['class_id'].values.tolist())

In [ ]:
weights = [1]*5
weights += [3]
weights1 = [3,2,4,5]
iou_thr = 0.25
skip_box_thr = 0.0
sigma = 0.1

fnl = {}

for i in tqdm(boxes_dict.keys()):
    
    
    boxes, scores, labels = weighted_boxes_fusion(boxes_dict[i], scores_dict[i], labels_dict[i],\
                                                  weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    
    
    
    
    
#     boxes1, scores1, labels1 = nms(boxes_dict[i], scores_dict[i], labels_dict[i], weights=weights, iou_thr=iou_thr)

    
    
    
    
    
#     boxes0, scores0, labels0 = soft_nms(boxes_dict[i], scores_dict[i], labels_dict[i], weights=weights,\
#                                      iou_thr=iou_thr, sigma=sigma, thresh=skip_box_thr)
    
#     boxes2, scores2, labels2 = non_maximum_weighted(boxes_dict[i], scores_dict[i], labels_dict[i], weights=weights,
#                                                                 skip_box_thr=skip_box_thr)

    
    
    
#     boxes, scores, labels = weighted_boxes_fusion([boxes0,boxes1,boxes2,boxes3],\
#                                                   [scores0,scores1,scores2,scores3],\
#                                                   [labels0,labels1,labels2,labels3],\
#                                                   weights=weights1, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    
    if not i in fnl.keys():
        fnl[i] = {'boxes':[],'scores':[],'labels':[]}
        
    fnl[i]['boxes'] = boxes*whwh_dict[i]
    fnl[i]['scores'] = scores
    fnl[i]['labels'] = labels

In [ ]:
pd_form = []
for i in fnl.keys():
    b = fnl[i]
    for j in range(len(b['boxes'])):
        pd_form.append([i,int(b['labels'][j]),round(b['scores'][j],2),\
                        int(b['boxes'][j][0]),int(b['boxes'][j][1]),\
                        int(b['boxes'][j][2]),int(b['boxes'][j][3])])
        
final_df = pd.DataFrame(pd_form,columns = ['image_id','class_id','score','x_min','y_min','x_max','y_max'])
final_df = final_df.drop_duplicates(keep = 'first')

In [ ]:
def submission_encoder(df:pd.DataFrame) -> np.ndarray:
    dct = {}
    for i in tqdm(df['image_id'].unique()):
        if not i in dct.keys():
            dct[i] = []
        tmp = df[df['image_id'] == i].values
        for j in tmp:
            dct[i].append(int(j[1]))
            dct[i].append(float(j[2]))
            dct[i].append(int(j[3]))
            dct[i].append(int(j[4]))
            dct[i].append(int(j[5]))
            dct[i].append(int(j[6]))
        
        dct[i] = map(str,dct[i])
        dct[i] = ' '.join(dct[i])
    dct = [[k, v] for k, v in dct.items()]
    return pd.DataFrame(dct,columns = ['image_id','PredictionString']).reset_index(drop = True)

df = submission_encoder(final_df)
df.to_csv('Fold5Yolo.csv', index=False)

In [ ]:
NORMAL = "14 1 0 0 1 1"
low_threshold = 0.00
high_threshold = 0.99
pred_det_df = df  # You can load from another submission.csv here too.
n_normal_before = len(pred_det_df.query("PredictionString == @NORMAL"))
merged_df = pd.merge(pred_det_df, pred_2cls, on="image_id", how="left")

if "target" in merged_df.columns:
    merged_df["class0"] = 1 - merged_df["target"]

c0, c1, c2 = 0, 0, 0
for i in range(len(merged_df)):
    p0 = merged_df.loc[i, "class0"]
    if p0 < low_threshold:
        # Keep, do nothing.
        c0 += 1
    elif low_threshold <= p0 and p0 < high_threshold:
        # Add, keep "det" preds and add normal pred.
        if ' 14 ' not in merged_df.loc[i, "PredictionString"]:
            merged_df.loc[i, "PredictionString"] += f" 14 {p0} 0 0 1 1"
            
        c1 += 1
    else:
        # Replace, remove all "det" preds.
        merged_df.loc[i, "PredictionString"] = NORMAL
        c2 += 1

n_normal_after = len(merged_df.query("PredictionString == @NORMAL"))
print(
    f"n_normal: {n_normal_before} -> {n_normal_after} with threshold {low_threshold} & {high_threshold}"
)
print(f"Keep {c0} Add {c1} Replace {c2}")
submission_filepath = str("submission.csv")
submission_df = merged_df[["image_id", "PredictionString"]]
submission_df.to_csv(submission_filepath, index=False)
print(f"Saved to {submission_filepath}")